<a href="https://colab.research.google.com/github/VellummyilumVinoth/Toxic_Comment_Classification/blob/main/predict_LSTM_without_subword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import load_model

In [3]:
# Load the saved model
model = load_model('/content/drive/MyDrive/LSTM_without_subword')

In [4]:
df1 = pd.read_csv('/content/drive/MyDrive/preprocessed_Reddit_Data_1.csv')
sentiment = df1['Title'].values


In [5]:
# Preprocess the new dataset df1
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(sentiment))  # Assuming sentiment is the training dataset
predict_data = df1['Title'].values
test_seq = tokenizer.texts_to_sequences(predict_data)
test_pad = pad_sequences(test_seq, maxlen=100)


In [6]:
# Make predictions on the new dataset
y_test_with_subword = model.predict([test_pad], batch_size=1024, verbose=1)

# Apply threshold and convert to 0 or 1
threshold = 0.5
y_test_with_subword[y_test_with_subword >= threshold] = 1
y_test_with_subword[y_test_with_subword < threshold] = 0
y_test_with_subword = y_test_with_subword.astype(int)

# Create a DataFrame with the predicted labels
y_list = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
predict_labels_df_with_subword = pd.DataFrame(y_test_with_subword, columns=y_list)

# Create a DataFrame with the original data from df1
predict_data_df = pd.DataFrame(predict_data, columns=["Title"])

# Combine the original DataFrame with the predicted labels
predict_df_with_subword = pd.concat([predict_data_df, predict_labels_df_with_subword], axis=1)

# Save the DataFrame to a CSV file
predict_df_with_subword.to_csv('/content/drive/MyDrive/predict_lstm_without_subword_reddit.csv', index=False)

# Print the predicted DataFrame
predict_df_with_subword


1/1 [==============================] - 4s 4s/step


,Title,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,UkrainianConflict Discussion Megathread,1,1,1,1,1,1
1,Zelenskyy survives over 12 assassination attem...,1,1,1,1,1,1
2,In the first round of presidential elections i...,1,1,1,1,1,1
3,"A further 20,000 Ukrainian recruits will be tr...",1,1,1,1,1,1
4,"Zelensky: ""Tanks, APCs and artillery are exact...",1,1,1,1,1,1
...,...,...,...,...,...,...,...
971,Hundreds of US military vehicles arrive in Dut...,1,1,1,1,1,1
972,BREAKING: Poland will deliver a company of Leo...,1,1,1,1,1,1
973,Russian airline aircraft suffer massive breakd...,1,1,1,1,1,1
974,"The Russian Federation declared that it ""has t...",1,1,1,1,1,1
